In [1]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from torchvision import transforms

from trainer import fit
import numpy as np

cuda = torch.cuda.is_available()

In [4]:
query_folder = '../AIC20_ReID/image_query'
gallery_folder = '../AIC20_ReID/image_test'
output_folder = '../query_results/'
model_path = 'weights/onlinetriplet-b4-200405-hardest_30epochs.pth'

In [5]:
# Prepare DataLoaders
from datasets import ImageFolderDataset
from pathlib import Path

query_names = []
for file_name in Path(query_folder).glob('*.jpg'):
    query_names.append(str(file_name.parts[-1]))
gallery_names = []    
for file_name in Path(gallery_folder).glob('*.jpg'):
    gallery_names.append(str(file_name.parts[-1]))

size = (224, 224)

query_dataset = ImageFolderDataset(query_folder, query_names, query_names,
                                       transform = transforms.Compose([
                                        transforms.Resize(size),  
                                        transforms.ToTensor()
                                      ]))
gallery_dataset = ImageFolderDataset(gallery_folder, gallery_names, gallery_names,
                                     transform = transforms.Compose([
                                        transforms.Resize(size),
                                        transforms.ToTensor()
                                      ]))

batch_size = 8
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
query_loader = torch.utils.data.DataLoader(query_dataset, batch_size=batch_size, shuffle=True, **kwargs)
gallery_loader = torch.utils.data.DataLoader(gallery_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [ ]:
# Load Model
model = torch.load(model_path)
# feature_extractor = model.embedding_net
feature_extractor = model

In [5]:
N_DIMS = 1792
def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), N_DIMS))
        labels = []
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels += target
            k += len(images)
    return embeddings, labels

In [8]:
query_embedding, query_labels = extract_embeddings(query_loader, model)
gallery_embedding, gallery_labels = extract_embeddings(gallery_loader, model)

In [9]:
import torch

def pdist_torch(emb1, emb2):
    m, n = emb1.shape[0], emb2.shape[0]
    emb1_pow = torch.pow(emb1, 2).sum(dim = 1, keepdim = True).expand(m, n)
    emb2_pow = torch.pow(emb2, 2).sum(dim = 1, keepdim = True).expand(n, m).t()
    dist_mtx = emb1_pow + emb2_pow
    dist_mtx = dist_mtx.addmm_(1, -2, emb1, emb2.t())
    dist_mtx = dist_mtx.clamp(min = 1e-12).sqrt()
    return dist_mtx

def run_query(emb_query, emb_gallery, top_k=100):
    #Calculate distance matrix between query images and gallery images
    dist_mtx = pdist_torch(emb_query,emb_gallery).cpu().detach().numpy()
    n_q, n_g = dist_mtx.shape
    #sort "gallery index" in "distance" ascending order 
    indices = np.argsort(dist_mtx, axis = 1)
    return indices[:, :top_k]

In [10]:
query_tensor = torch.from_numpy(query_embedding)
gallery_tensor = torch.from_numpy(gallery_embedding)
if cuda:
    query_tensor = query_tensor.cuda()
    gallery_tensor = gallery_tensor.cuda()
    
indices = run_query(query_tensor, gallery_tensor)

In [23]:
import os

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
for i, query_file in enumerate(query_labels):
    file_path = os.path.join(output_folder, os.path.splitext(query_file)[0] + ".txt")
    with open(file_path, "w") as f:
        for j in range(100):
            f.write(os.path.splitext(gallery_labels[indices[i][j]])[0] + "\n")
    if (i + 1 % 100 == 0):
        print("PROCESSED %d FILES" % (i + 1))
